In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

y = np.random.normal(0,1.5,1000).cumsum()

plt.figure(figsize=(10,6))
plt.plot(y)
plt.title('Super Big Random Walk')
plt.show()

In [ ]:
#turn the series into a suitable sf dataframe
series = pd.DataFrame({'y': y, 'unique_id': 0, 'ds': range(0, len(y))})
series

In [ ]:
from statsforecast.core import StatsForecast
from statsforecast.models import ( 
    AutoARIMA,
    AutoETS
)


sf = StatsForecast(
    models=[AutoETS(season_length=7)],
    freq=1
)

#sf.forecast(df=series, h=7).head() #to see if it works

In [ ]:
import ray
import logging

ray.init(logging_level=logging.ERROR)

series = series.reset_index()
series['unique_id'] = series['unique_id'].astype(str)
ctx = ray.data.context.DatasetContext.get_current()
ctx.use_streaming_executor = False
ray_series = ray.data.from_pandas(series).repartition(4)

sf.forecast(df=ray_series, h=7).take(5)